In [ ]:
#!pip install requests -U
#!pip install html5lib -U
#!pip install bs4 -U

In [ ]:
import pandas as pd
import yfinance as yf
import pandas as pd

import numpy as np 
import math
import matplotlib. pyplot as plt 
import seaborn as sns 
from scipy import stats


In [ ]:
import os
import glob

def delete_files_in_directory(directory_path):
   try:
     files = glob.glob(os.path.join(directory_path, '*.csv'))
     for file in files:
       if os.path.isfile(file):
         os.remove(file)
     print("All files deleted successfully.")
   except OSError:
     print("Error occurred while deleting files.")

# Usage
directory_path = '/Users/marcellofelipelli/Documents/DataAnalyticsPortfolio/Finance/Data'
delete_files_in_directory(directory_path)

In [ ]:
tickers = ['WEGE3.SA','AURE3.SA','LEVE3.sa','CSMG3.SA'
           ,'PETR4.SA','PETR3.SA','MTRE3.SA','BRAP4.SA','BRAP3.SA'
           ,'GOAU4.SA','AGRO3.SA','SLCE3.SA'
           ,'VALE3.SA','CPFE3.SA','SBSP3.SA'
           ,'ITUB3.SA','ITUB4.SA','BMGB4.SA']

In [ ]:
import requests 
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

for ticket in tickers:
    URL = 'https://www.fundamentus.com.br/detalhes.php?papel={}'.format(ticket.split('.')[0]) 
    # r = requests.get(URL)
    print(URL) 

    site= URL
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    tables = soup.find_all('table')  
    tbl_dados = []
    tbl_label = []

    data_dict = {}
    #print(tables)
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            label = row.find_all('td',{'class':'label'})
            dado = row.find_all('td',{'class':'data'})

            label_txt = ''
            dado_txt = ''

            for i in range(len(label)):
                try:
                    label_txt = label[i].find('span',{'class':'txt'}).text.replace('\n', ' ').replace('\r', '')
                except:
                    label_txt = ''

                try:    
                    dado_txt = dado[i].find('span',{'class':'txt'}).text.replace('\n', ' ').replace('\r', '')        
                except:
                    try:    
                        dado_txt = dado[i].find('span',{'class':'oscil'}).text.replace('\n', ' ').replace('\r', '')        
                    except:
                        dado_txt = ''

                data_dict.update({label_txt : dado_txt})

    
    df = pd.DataFrame(data_dict.items())    

    df['Ticker'] = ticket

    df.to_csv('Data/dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
    try:
        stock = yf.Ticker(ticket)
        df = stock.history(period="5y")
        df.to_csv('Data/historico_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        # df['Close'].plot(title="{}'s stock price".format(ticket))
        stock.recommendations.to_csv('Data/recomentations_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        stock.actions.to_csv('Data/actions_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        stock.dividends.to_csv('Data/dividendos_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        pd.DataFrame(stock.info).to_csv('Data/info_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        stock.cashflow.to_csv('Data/cashflow_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        stock.financials.to_csv('Data/financial_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
        pd.DataFrame(stock.calendar).to_csv('Data/calendario_dados_ticket_{}.csv'.format(ticket.split('.')[0]), index=True)
    except:
        print(ticket)    


